In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import abatement, techdata_to_tree, sys, ShockFunction
os.chdir(directory['curr'])
def flatten_list(list_):
    return [item for sublist in list_ for item in sublist]

The file_gams_py_gdb0.gdx is still active and was not deleted.


## 1: Set up tree

*Read data:*

In [2]:
inputfile = "techdata_dors_3_overlap.xlsx" # file
inputoth  = 'Othdata_dors_3.xlsx'
output = techdata_to_tree.load_techcats(pd.read_excel(directory['data'] + "/" + inputfile, sheet_name=["inputdisp", "endofpipe", "inputprices"]))
modules = ["ID", "EOP"] # modules to use

*Set up trees:*

In [3]:
nts = {m: nesting_tree.nesting_tree(name=m) for m in modules}

#### 1.1: Input displacing tree

In [4]:
nts["ID"].add_tree(output["ID"]["upper_categories"], tree_name = 'ID_EC', **{"type_f":"CES_norm"}) # E to C
nts["ID"].add_tree(output["ID"]["components"], tree_name = "ID_CU", **{"type_f":"MNL"}) # C to U
nts["ID"].add_tree(output["ID"]["techs"], tree_name="ID_TU", **{'type_io': 'output', 'type_f': 'CET_norm'}) # U to T
nts["ID"].add_tree(output["ID"]["techs_inputs"], tree_name="ID_TX") # T to inputs X
nts["ID"].add_tree(output["ID"]["basetechs"], tree_name="ID_BU", **{"type_io":"output", "type_f":"linear_out"}) # baseline technologies
nts["ID"].add_tree(output["ID"]["basetech_inputs"], tree_name="ID_BX")

#### 1.2: End of pipe tree

In [5]:
nts["EOP"].add_tree(output["EOP"]["components"], tree_name = "EOP_CU", **{"type_f":"MNL"}) # C to U
nts["EOP"].add_tree(output["EOP"]["techs"], tree_name="EOP_TU", **{'type_io': 'output', 'type_f': 'CET_norm'}) # T to U 
nts["EOP"].add_tree(output["EOP"]["techs_inputs"], tree_name="EOP_TX") # T to inputs X

#### 1.3: Trees related to final goods and emissions account

*Read in tree and data from xlsx:*

In [6]:
nts['ID'].add_tree(directory['data']+'\\'+inputoth,tree_name='ID_Y',**{'sheet':'Y'})
# DataBase.GPM_database.merge_dbs(nts['ID'].trees['ID_Y'].database,
#                                 excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'Y': 'vars', 'M': 'vars','M_sets': 'sets'}), 'first')


DataBase.GPM_database.merge_dbs(nts['ID'].trees['ID_Y'].database,
                                excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'Y': 'vars'}), 'first')

*Add new Q2P:*

In [7]:
output['ID']['Q2P'] = output['ID']['Q2P'].append(excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'Q2P': 'maps'}).database['Q2P'].vals)

In [8]:
#midlertidigt fix. run_all metoden laver nodes som ikke er i træet om til nan her i phi
#phi = nts["ID"].trees["ID_Y"].database["phi"].vals.copy()

#### 1.3: Namespaces

*Give modules local namespaces:*

In [9]:
standard_sets = ('inp','out','int','wT','map_all','kno_out','kno_inp')
for module in modules:
    namespace = {k: module + '_' + k for k in standard_sets}
    nts[module].run_all(**namespace)
    #Also replaces keys with module-specific names, i.e. changes 'inp' to 'ID_inp' in the attributes/keys.
    for std_set in standard_sets:
        if hasattr(nts[module], std_set):
            setattr(nts[module], module + "_" + std_set, getattr(nts[module], std_set))
            delattr(nts[module], std_set)

In [10]:
# midlertidigt fix af phi der giver naner på alt andet end inp3
DataBase.GPM_database.merge_dbs(nts['ID'].database,
                                excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'M': 'vars','M_sets': 'sets'}), 'first')

#### 1.4 Additional objects derived from the catalog

*Load*: 
- $\mu$ parameters deducted from the catalog directly, 
- current applications according to the potential split (latter used for starting values),
- Q2P



## 2: Initialize model

Initialize:

In [12]:
gm = abatement.abate(nt=nts['ID'],tech=output,work_folder=directory['work'],**{'data_folder':directory['gams'],'name': 'atest'})

In [13]:
gm.add_aliases([('n','nnnn'),('n','nnnnn')])
gm.model.database.update_all_sets(clean_up=False)

In [15]:
gm.write_and_run(kwargs_init={'check_variables':True})

In [16]:
db_star = DataBase.GPM_database(**{'name': 'db_star'})
sigma_star = gm.get("sigma").copy()
eta_star = gm.get("eta").copy()

In [17]:
eta_star[:] = -3
sigma_star[gm.get("kno_ID_CU")] = 0.4
sigma_star[gm.get("kno_no_ID_BX")] = 0.5
sigma_star["Y"] = 0.2

In [18]:
db_star["sigma"] = sigma_star
db_star["eta"] = eta_star

In [19]:
name='v0'
gm.write_and_run(name=name, add_checkpoint=name)
gm.checkpoints['v1'] = gm.model_instances[name].ws.add_checkpoint()

In [20]:
gm.model_instances[name].solve_sneakily(db_star=db_star,from_cp = True, cp_init = gm.checkpoints[name], options_run={'checkpoint': gm.checkpoints['v1']})

{'Modelstat': 16.0, 'Solvestat': 1.0}

In [21]:
gm.model.database.merge_dbs(gm.model.database,gm.model_instances['v0'].out_db,'second')

In [22]:
db_star = DataBase.GPM_database(**{'name': 'db_star'})
sigma_star = gm.get("sigma").copy()
sigma_star[gm.get("kno_ID_CU")] = 4
db_star["sigma"] = sigma_star

In [23]:
name='v2'
gm.write_and_run(name=name, add_checkpoint=name)

In [25]:
gm.checkpoints['v3'] = gm.model_instances[name].ws.add_checkpoint()

In [27]:
gm.model_instances[name].solve_sneakily(db_star=db_star,from_cp = True, cp_init = gm.checkpoints[name], options_run={'checkpoint': gm.checkpoints['v3']})

{'Modelstat': 16.0, 'Solvestat': 1.0}

In [7]:
gm = Production.pr_static(nt=nt,work_folder=directory['work'],**{'data_folder':gams_folder,'name':'A1'})
gm.write_and_run(kwargs_init={'check_variables':True})

In [9]:
dbs = {"ID":DataBase.GPM_database(), "EOP":DataBase.GPM_database()}

*Define auxiliary sets:*

In [10]:
m = 'ID'
db,nt = nts[m].database, nts[m]
n,nn,nnn = nts[m].n, nts[m].nn, nts[m].nnn

*Subsets for different types of nests:*

In [11]:
db[m+'_t'] = pd.Index(output[m]['techs'].keys(),name=n) # technologies
db[m+'_t_all'] = pd.Index(output[m]['basetechs'].keys(),name=n).union(db.get(m+'_t')) # technologies including baselines
db[m+'_c'] = pd.Index(output[m]['components'].keys(), name = n) # subset of components
db[m+'_e'] = pd.Index(output[m]['upper_categories'].keys(), name = n) # energy services
db[m+'_e_all'] = pd.Index(['Y'],name=n).union(db.get(m+'_e')) # energy services nest including other inputs (Y)
db[m+'_u_all'] = pd.Index([x for y in output[m]['components'].values() for x in y], name = n) # all technology goods
db[m+'_u'] = db.get(m+'_u_all').difference(pd.Index([x for y in output[m]['basetechs'].values() for x in y], name = n)) # technology goods, not baseline firms
db['ai'] = output['ID']['Q2P'].levels[1].rename(n) # subset of aggregate goods.

*Mappings (do we want to store these for later?):*

In [14]:
i2t = db.get(m+'_map_all')[(db.get(m+'_map_all').get_level_values(n).isin(db.get(m+'_inp'))) & 
                           (db.get(m+'_map_all').get_level_values(nn).isin(db.get(m+'_t_all')))]
u2t = db.get(m+'_map_all')[(db.get(m+'_map_all').get_level_values(n).isin(db.get(m+'_u_all'))) &
                           (db.get(m+'_map_all').get_level_values(nn).isin(db.get(m+'_t_all')))]
u2c = db.get(m+'_map_all')[(db.get(m+'_map_all').get_level_values(n).isin(db.get(m+'_u_all'))) & 
                           (db.get(m+'_map_all').get_level_values(nn).isin(db.get(m+'_c')))]
c2e = db.get(m+'_map_all')[(db.get(m+'_map_all').get_level_values(n).isin(db.get(m+'_c'))) & 
                           (db.get(m+'_map_all').get_level_values(nn).isin(db.get(m+'_e_all')))]

*When identifying the level of current coverage for a technology type, we map from energy service to u:*

In [15]:
db[m+'_e2u'] = DataBase_wheels.appmap(u2c,DataBase_wheels.map_from_mi(c2e,n,nn),nn).swaplevel(0,1).set_names([n,nn])

*Energy service type to technology type*

In [16]:
db[m+'_e2t'] = DataBase_wheels.appmap(db.get(m+'_e2u'),DataBase_wheels.map_from_mi(u2t,n,nn),nn).unique()

*Energyservice 2 aggregate input type 2 inidividual input type:*

In [25]:
pd.MultiIndex.from_product([pd.Index(['CO2'],name='z'),db.get('ai')])

MultiIndex([('CO2',           'K'),
            ('CO2', 'electricity'),
            ('CO2',        'inp3'),
            ('CO2',         'oil')],
           names=['z', 'n'])

In [17]:
i2ai = output['ID']['Q2P'] # individual inputs 2 aggregate inputs
t2i2ai = DataBase_wheels.mi.add_ndmi(i2t.swaplevel(0,1).set_names([n,nn]),i2ai.set_names([nn,nnn])) # technology 2 individual 2 aggregate input
db[m+'_e2ai2i'] = DataBase_wheels.appmap(t2i2ai,DataBase_wheels.map_from_mi(db.get(m+'_e2t'),nn,n),n).swaplevel(1,2).set_names([n,nn,nnn])

With these mappings:
* Define qSumU[n,nn] as the production by technology (n) of energy service (nn). This should be computed as: (NOTE: We define qsumU over n=e and nn = t instead):

$$
\begin{align}
    \text{qSumU[n,nn]} = \text{sum(nnn\$(e2t[nn,n] and e2u[nn,nnn]), qD[nnn])}.
\end{align}
$$

* Define the output share os[n,nn] as the share of output that technology $n$ produces of energy-service type $nn$. This should be defined as:

$$
\begin{align}
    \text{os[n,nn]} = \text{sum(nnn\$(e2u[nn,nnn] and u2t[nnn,n]), qD[nnn]) / qD[n]}.
\end{align}
$$

* Define variable qSumX[n,nn] defined as the use of aggregate input type $nn\in (electricity,oil,inp3,K)$ used to produce energy service $n$. This is defined as:

$$\begin{align}
    \text{qSumX[n,nn]} = \text{sum([nnn,nnnn]\$(e2ai2i[n,nn,nnn] and e2t[n,nnnn] and i2t[nnn,nnnn]), qD[nnn] * os[nnnn,n])}
\end{align}$$
* Here e2ai2i identifies the relevant quantities $qD[i]$ that are tied to relevant aggregate type of input, $e2t[e,t]$ identifies the correct output-share $os[t,e]$, and $i2t[i,t]$ ensures that only the relevant $i,t$ combinations are summed over.


Beyond these book keeping variables, we define a number of variables used in calibration:

* Define $currapp[n,nn]$ as the current share of coverage of a technology $n$ of energy-service $nn$. This is simply defined as (maybe we don't need to define the qSumU variable?)

$$\begin{align}
    curapp[n,nn] = qSumU[n,nn] / qD[nn].
\end{align}$$
* Define the level of application ... 